# AVISO+ 기반 사르가숨(Sargassum) 부유 조류 지수 분석 실습

AVISO+에서 제공하는 정규 부유 조류 지수(NFAI: Normalized Floating Algae Index) 자료를 활용하여, 특정 지역의 시계열 사르가숨 분석을 수행하는 실습 예제입니다.  
본 실습에서는 데이터 다운로드부터 시계열 지도 시각화, 그리고 날짜별 조류 지수 총합 분석까지 단계별로 살펴봅니다.

In [1]:
import requests, os
import xarray as xr
import numpy as np
import leafmap
import rasterio
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from pathlib import Path
from datetime import datetime

## AVISO+ NCSS 설정 및 분석 영역 정의

여기에서는 AVISO+ NCSS API 요청을 위해 필요한 URL, 사용자 인증 정보, 분석 영역(Bounding Box), 분석 기간을 설정합니다.  
또한 사용할 변수 목록(nfai_mean, nfai_max, nfai_min, nfai_nbpts)을 정리합니다.

In [2]:
# NCSS(NetCDF Subset Service) 요청 URL
NCSS_URL = "https://tds-odatis.aviso.altimetry.fr/thredds/ncss/grid/dataset-sargassum-cls-merged-msi-oli-global-lr"

# 사르가숨(Sargassum) 탐지 산출물
# AVISO+에 미등록된 경우: https://www.aviso.altimetry.fr/en/data/data-access/registration-form.html
# AVISO+에 이미 등록한 경우: https://www.aviso.altimetry.fr/en/my-aviso-plus.html

# 인증 정보
# 아래 'your-username', 'your-password' 부분을 실제 AVISO 계정 정보로 변경하세요.
USER = os.getenv("AVISO_USER", "bhyu@knps.or.kr")
PW   = os.getenv("AVISO_PW",   "SgeEoU")

# 분석 영역(Bounding Box)
west, east, south, north = -68.92405248166772, -68.40695965660555, 17.99922814815082, 18.479964791218272
BBOX = dict(north=north, south=south, east=east, west=west)

# 분석 기간
YEAR, MONTH = 2025, 7
time_start = f"{YEAR}-{MONTH:02d}-01T00:00:00Z"
time_end   = f"{YEAR}-{MONTH:02d}-31T23:59:59Z"

# 요청 변수 목록
# nfai_mean  : bin_size° 격자 셀에서 정규 부유 조류 지수(NFAI) 평균값의 일일 평균
# nfai_max   : bin_size° 격자 셀에서 정규 부유 조류 지수(NFAI) 평균값의 일일 최대값
# nfai_min   : bin_size° 격자 셀에서 정규 부유 조류 지수(NFAI) 평균값의 일일 최소값
# nfai_nbpts : bin_size° 격자 셀에서 정규 부유 조류 지수(NFAI) 픽셀 값 개수
VARS = ["nfai_mean", "nfai_max", "nfai_min", "nfai_nbpts"]

## NetCDF 다운로드 함수 정의

아래의 함수는 AVISO+ NCSS 서버에 GET 요청을 보내 NetCDF 파일을 다운로드하는 기능을 수행합니다.  
요청 파라미터 생성, 인증(Session), 응답 형식 검증, 파일 저장 과정 등을 포함합니다.

In [3]:
def fetch_ncss(url: str, out_path):
    """Download NetCDF from NCSS URL for a given area/period/variables"""
    params = {
        "var": VARS,
        "north": BBOX["north"],
        "south": BBOX["south"],
        "east":  BBOX["east"],
        "west":  BBOX["west"],
        "horizStride": 1,
        "time_start": time_start,
        "time_end":   time_end,
        "accept": "netcdf4",
    }

    with requests.Session() as s:
        s.auth = (USER, PW)
        r = s.get(url, params=params, timeout=180)
        r.raise_for_status()
        ctype = r.headers.get("Content-Type", "")
        if "netcdf" not in ctype.lower():
            raise RuntimeError(f"The server returned a response that is not NetCDF: {ctype}")
        out_path.write_bytes(r.content)

    return out_path

## NetCDF 데이터 다운로드

앞서 정의한 함수를 이용하여 실제 NetCDF 파일을 다운로드하고, 올바르게 저장되었는지 확인합니다.

In [4]:
nc_path = Path(f"{YEAR}_{MONTH:02d}_nfai.nc")
saved = fetch_ncss(NCSS_URL, nc_path)
print(f"[Save complete] {saved}")

[Save complete] 2025_07_nfai.nc


## NetCDF 파일 구조 확인

xarray를 활용하여 NetCDF 파일 내부 구조를 확인합니다.  
여기에서는 다음 항목들을 점검합니다.

- 전체 변수 구조
- time, latitude, longitude 차원 크기
- 날짜(time dimension) 목록
- 위도·경도 해상도 정보

이 정보는 추후 GeoTIFF 변환과 시계열 분석의 기반이 됩니다.

In [5]:
# NetCDF 파일 열기
ds = xr.open_dataset(nc_path, engine="h5netcdf")

print("=== Dataset overview ===")
print(ds)

# 시간 차원(Time dimension) 확인
time_dim = None
for cand in ["time", "Time", "t"]:
    if cand in ds.dims:
        time_dim = cand
        break

if time_dim:
    n_layers = ds.sizes[time_dim]
    print(f"\n이 파일에는 총 {n_layers}개의 시간 레이어가 포함되어 있습니다.")
    print("시간 목록:", ds[time_dim].values)
else:
    print("\n이 파일에는 시간 차원이 존재하지 않습니다.")

ds.close()

=== Dataset overview ===
<xarray.Dataset> Size: 36MB
Dimensions:     (time: 32, latitude: 193, longitude: 208)
Coordinates:
  * time        (time) datetime64[ns] 256B 2025-07-01 2025-07-02 ... 2025-08-01
  * latitude    (latitude) float64 2kB 18.0 18.0 18.0 ... 18.48 18.48 18.48
  * longitude   (longitude) float64 2kB -68.92 -68.92 -68.92 ... -68.41 -68.41
Data variables:
    nfai_mean   (time, latitude, longitude) float64 10MB ...
    nfai_max    (time, latitude, longitude) float64 10MB ...
    nfai_min    (time, latitude, longitude) float64 10MB ...
    nfai_nbpts  (time, latitude, longitude) float32 5MB ...
Attributes: (12/23)
    cdm_data_type:            Grid
    instrument:               MSI, OLI
    platform:                 Sentinel-2A, Sentinel 2-B, Landsat-8 and Landsat-9
    summary:                  This dataset provides statistics of the normali...
    comment:                  MSI data are provided by ESA through the Copern...
    institution:              Collecte Locali

## 특정 날짜의 원본 데이터 확인

nfai_nbpts 또는 다른 변수를 선택하여 특정 날짜(time slice)의 통계치를 확인합니다.  
최소값, 최대값, nodata 값 유무, NaN 개수를 검토하여 데이터 이상 여부를 파악합니다.

In [6]:
with xr.open_dataset(nc_path, engine="h5netcdf") as ds:
    var_name = "nfai_nbpts"
    date_str = "2025-07-10"
    date = np.datetime64(date_str)

    da_raw = ds[var_name].sel(time=date)

    print("원본 데이터 최소/최대값:")
    print("min:", float(da_raw.min().values))
    print("max:", float(da_raw.max().values))

    # 혹시 NaN이 많은지도 확인
    print("NaN 개수:", int(np.isnan(da_raw.values).sum()))

원본 데이터 최소/최대값:
min: -5000.0
max: 1.0
NaN 개수: 23830


## NetCDF → GeoTIFF 변환 함수 정의

아래 함수는 특정 날짜의 변수를 선택하여 GeoTIFF로 변환하고 저장합니다.

처리 과정은 다음과 같습니다.

- 날짜 선택
- nodata 값 마스킹
- 좌표계(EPSG:4326) 적용
- latitude/longitude → y/x 명칭 통일
- rioxarray 기반 GeoTIFF 저장

저장 파일명은 `{변수명}_{날짜}.tif` 형식으로 생성됩니다.

In [7]:
def nfai_nc_to_tiff(nc_path, var_name, date_str, out_dir=".", default_nodata=-5000.0):
    """
    NFAI 변수를 NetCDF → GeoTIFF로 변환하여 저장하는 함수.

    Parameters
    ----------
    nc_path : str or Path
        NetCDF 파일 경로
    var_name : str
        'nfai_mean', 'nfai_max', 'nfai_min', 'nfai_nbpts' 중 하나
    date_str : str
        'YYYY-MM-DD' 형식 날짜
    out_dir : str or Path
        GeoTIFF 출력 경로
    default_nodata : float
        NetCDF 내에 _FillValue가 없는 경우 사용할 nodata 값

    Returns
    -------
    Path : 생성된 GeoTIFF 파일 경로
    """

    date = np.datetime64(date_str)
    nc_path = Path(nc_path)
    out_dir = Path(out_dir)

    with xr.open_dataset(nc_path, engine="h5netcdf") as ds:
        nodata = float(ds[var_name].attrs.get("_FillValue", default_nodata))

        da = (
            ds[var_name]
            .sel(time=date)
            .where(lambda x: x != nodata)
            .rename(latitude="y", longitude="x")
            .sortby("y", ascending=False)
            .rio.write_crs("EPSG:4326")
            .rio.write_nodata(nodata)
            .fillna(nodata)
            .load()
        )

        # 밴드 이름 설정
        da.name = var_name

    # 저장 경로
    tif_path = out_dir / f"{var_name}_{date_str}.tif"

    # GeoTIFF 저장
    da.rio.to_raster(tif_path, overwrite=True)

    print(f"[Saved] {tif_path}")

## 특정 날짜에 대한 GeoTIFF 변환 실행

정의된 변환 함수를 사용하여 예시 날짜(예: 2025-07-10)에 대한 GeoTIFF를 생성하고, 생성된 파일 경로를 확인합니다.

In [8]:
nfai_nc_to_tiff("2025_07_nfai.nc", "nfai_nbpts", "2025-07-10")

[Saved] nfai_nbpts_2025-07-10.tif


## Leafmap 단일 시각화

변환한 GeoTIFF 파일을 Leafmap을 이용하여 시각화합니다.  
Esri 또는 Google 위성지도 위에 NFAI 데이터를 덧씌워 확인할 수 있습니다.

In [9]:
tif_path = "nfai_nbpts_2025-07-10.tif"

# 지도 생성
m = leafmap.Map(width="700px", height="400px", zoom=10)
m.add_basemap("Esri.WorldImagery")
m.add_raster(
    str(tif_path),
    colormap="viridis",
    vmin=0, vmax=1,
    nodata=-5000.0,
    opacity=1.0,
    layer_name=f"{var_name} {date_str}",
    zoom_to_layer=True,
)

m

Map(center=[18.240000000000002, -68.66624999999999], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## 전체 날짜 목록 추출

NetCDF의 time dimension을 읽어 YYYY-MM-DD 형태의 날짜 문자열로 변환합니다.  
이 날짜 목록은 TIFF 파일 자동 생성 및 시간 슬라이더 지도 생성에 사용됩니다.

In [10]:
# nc 파일에서 날짜 목록 확인
with xr.open_dataset(nc_path, engine="h5netcdf") as ds:
    times = ds["time"].values

start_date_str = np.datetime_as_string(times[0], unit="D")
end_date_str   = np.datetime_as_string(times[-1], unit="D")

print("시작 날짜:", start_date_str)
print("끝   날짜:", end_date_str)
print("총 time 스텝 수:", len(times))

시작 날짜: 2025-07-01
끝   날짜: 2025-08-01
총 time 스텝 수: 32


## 전체 날짜 GeoTIFF 자동 생성

time dimension을 순회하여 날짜별 TIFF 파일을 자동으로 생성합니다.  
출력되는 GeoTIFF 파일은 출력 폴더(out_dir)에 저장되며, 시계열 지도를 위해 사용할 수 있습니다.

In [11]:
nc_path = "2025_07_nfai.nc"
var_name = "nfai_nbpts"
out_dir = Path("nfai_nbpts")
out_dir.mkdir(exist_ok=True)

# 날짜별로 GeoTIFF 저장
for t in times:
    date_str = np.datetime_as_string(t, unit="D")
    nfai_nc_to_tiff(
        nc_path=nc_path,
        var_name=var_name,
        date_str=date_str,
        out_dir=out_dir,
        default_nodata=-5000.0,
    )

[Saved] nfai_nbpts\nfai_nbpts_2025-07-01.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-02.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-03.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-04.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-05.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-06.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-07.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-08.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-09.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-10.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-11.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-12.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-13.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-14.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-15.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-16.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-17.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-18.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-19.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-20.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-21.tif
[Saved] nfai_nbpts\nfai_nbpts_2025-07-22.tif
[Saved] nf

In [12]:
# 날짜 리스트
dates = ds["time"].dt.strftime("%Y-%m-%d").values.tolist()
print(dates)

['2025-07-01', '2025-07-02', '2025-07-03', '2025-07-04', '2025-07-05', '2025-07-06', '2025-07-07', '2025-07-08', '2025-07-09', '2025-07-10', '2025-07-11', '2025-07-12', '2025-07-13', '2025-07-14', '2025-07-15', '2025-07-16', '2025-07-17', '2025-07-18', '2025-07-19', '2025-07-20', '2025-07-21', '2025-07-22', '2025-07-23', '2025-07-24', '2025-07-25', '2025-07-26', '2025-07-27', '2025-07-28', '2025-07-29', '2025-07-30', '2025-07-31', '2025-08-01']


In [13]:
# 이미지 경로 리스트
images = [str(out_dir / f"{var_name}_{d}.tif") for d in dates]

print(images)

['nfai_nbpts\\nfai_nbpts_2025-07-01.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-02.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-03.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-04.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-05.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-06.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-07.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-08.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-09.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-10.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-11.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-12.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-13.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-14.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-15.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-16.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-17.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-18.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-19.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-20.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-21.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-22.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-23.tif', 'nfai_nbpts\\nfai_nbpts_2025-07-24.tif', 'nfai_nbpts\\nf

## Leafmap 시간 슬라이더(Time Slider) 기반 시계열 지도 생성

여기에서는 Leafmap의 add_time_slider 기능을 사용하여 날짜별 GeoTIFF를 지도에서 시계열로 재생할 수 있도록 구성합니다.  
사용자가 슬라이더를 이동하면 특정 날짜의 사르가숨 지수 TIFF가 자동으로 갱신됩니다.

In [ ]:
m = leafmap.Map(projection="globe", width="700px", height="400px")
m.add_basemap("Esri.WorldImagery")
m.add_time_slider(
    images,
    labels=dates,
    colormap="viridis",
    vmin=0, vmax=1,
    nodata=-5000.0,
    opacity=1.0,
    zoom_to_layer = True
)
m

## 날짜별 nfai_nbpts 총합 계산

NetCDF 파일에서 nfai_nbpts의 날짜별 총합을 계산합니다.

- nodata 제외
- 위도 x 경도 방향으로 합산
- 날짜별 총량(daily_sum) 생성

이 총합은 사르가숨의 상대적 분포 변화를 정량적으로 파악하는 데 사용할 수 있습니다.

In [ ]:
# nc 파일에서 날짜별 nfai_nbpts 총합 계산
with xr.open_dataset(nc_path, engine="h5netcdf") as ds:
    var_name = "nfai_nbpts"
    da = ds[var_name]

    # nodata 값 가져오기 (없으면 -5000.0 사용)
    nodata = float(da.attrs.get("_FillValue", -5000.0))

    # nodata를 제외한 유효값만 사용
    da_valid = da.where(da != nodata)

    # 위도/경도 방향으로 합산 → 날짜별 총 픽셀 수
    daily_sum = da_valid.sum(dim=["latitude", "longitude"])

    # x축에 쓸 날짜 문자열
    dates = ds["time"].dt.strftime("%Y-%m-%d").values

vals = daily_sum.values

# Normalize & Colormap
norm = colors.Normalize(vmin=vals.min(), vmax=vals.max())
cmap = cm.get_cmap("viridis")
colors_list = cmap(norm(vals))

# Figure & Axes 생성
fig, ax = plt.subplots(figsize=(10, 4))

# Bar Plot
bars = ax.bar(dates, vals, color=colors_list)

# 값이 있는 날짜만 라벨 표시
xtick_labels = [d if v > 0 else "" for d, v in zip(dates, vals)]

ax.set_xticks(range(len(dates)))
ax.set_xticklabels(xtick_labels, rotation=90, ha="right")

ax.set_xlabel("Date")
ax.set_ylabel("Total nfai_nbpts")
ax.set_title("Daily total nfai_nbpts")

# Colorbar
cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax)

plt.tight_layout()
plt.show()